In [ ]:
!pip install arize-phoenix
!pip install openinference-instrumentation-openai

## Setting up Arize Phoenix

https://docs.arize.com/phoenix/quickstart

### Command Line
```bash
python3 -m phoenix.server.main serve
```

### Docker
Launch the phoenix docker image using:
```bash
docker run -p 6006:6006 -p 4317:4317 arizephoenix/phoenix:latest
```
This will expose the Phoenix UI and REST API on localhost:6006 and exposes the gRPC endpoint for spans on localhost:4317

### Notebook
```python
import phoenix as px
px.launch_app()
```

In [1]:
import json
import os
from getpass import getpass
import nest_asyncio
import pandas as pd
from tqdm import tqdm

import phoenix as px
nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

In [2]:
import phoenix as px
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
(session := px.launch_app()).view()

In [ ]:
endpoint = "http://127.0.0.1:6006/v1/traces"

### Trace Open AI

For more integration - https://docs.arize.com/phoenix/tracing/integrations-tracing

In [ ]:
# !pip install openai

In [ ]:
import openai

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

tracer_provider = trace_sdk.TracerProvider()
span_exporter = OTLPSpanExporter(endpoint)
span_processor = SimpleSpanProcessor(span_exporter)
tracer_provider.add_span_processor(span_processor)
trace_api.set_tracer_provider(tracer_provider)

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument()

In [ ]:
client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Write a haiku."}],
)
print(response.choices[0].message.content)

## Trace Llama index

In [ ]:
# !pip install llama-index
# !pip install llama-index-core
# !pip install llama-index-llms-openai
# !pip install openinference-instrumentation-llama-index==2.2.4
# !pip install -U llama-index-callbacks-arize-phoenix
# !pip install "arize-phoenix[llama-index]"

In [ ]:
px.active_session().url

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")
resp = llm.complete("Paul Graham is ")
print(resp)

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

llm = OpenAI()
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

## Observe RAG Pipeline

In [ ]:
# !pip install llama-index
# !pip install llama-index-vector-stores-qdrant 
# !pip install llama-index-readers-file 
# !pip install llama-index-embeddings-fastembed 
# !pip install llama-index-llms-openai
# !pip install -U qdrant_client fastembed

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.fastembed import FastEmbedEmbedding
# from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core.settings import Settings

Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")
# Settings.embed_model = OpenAIEmbedding(embed_batch_size=10)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)